## taskflow 详解

In [1]:
#!/usr/bin/env python
# coding=utf-8

from __future__ import print_function
import taskflow.engines
from taskflow.patterns import linear_flow as lf
from taskflow import task

def flow_watch(state, details):
    print("Flow State:{}".format(state))
    print("Flow Details:{}".format(details))

class A(task.Task):
    def execute(self, a_msg, *args, **kwargs):
        print('A:{}' . format(a_msg))

class B(task.Task):
    def execute(self, b_msg, *args, **kwargs):
        print('B : {}' . format(b_msg))

flow = lf.Flow('simple-linear-listen').add(
    A(),
    B()
    )

engine = taskflow.engines.load(flow, store = dict(a_msg = 'a', b_msg = 'b'))

engine.notifier.register('*', flow_watch)

engine.run()


Flow State:RUNNING
Flow Details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c4516d0>, 'old_state': 'PENDING', 'flow_name': u'simple-linear-listen', 'flow_uuid': '4c07a097-4816-4be9-8fa5-e5ad19e7635e'}
A:a
B : b
Flow State:SUCCESS
Flow Details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c4516d0>, 'old_state': 'RUNNING', 'flow_name': u'simple-linear-listen', 'flow_uuid': '4c07a097-4816-4be9-8fa5-e5ad19e7635e'}


In [ ]:
### 监听任务

In [3]:
#!/usr/bin/env python
# coding=utf-8

from __future__ import print_function
import taskflow.engines
from taskflow.patterns import linear_flow as lf
from taskflow import task

def flow_watch(state, details):
    print("Flow State:{}".format(state))
    print("Flow Details:{}".format(details))

def task_watch(state, details):
    print('Task state:{}'.format(state))
    print('Task details:{}'.format(details))

class A(task.Task):
    def execute(self, a_msg, *args, **kwargs):
        print('A:{}' . format(a_msg))

class B(task.Task):
    def execute(self, b_msg, *args, **kwargs):
        print('B : {}' . format(b_msg))

flow = lf.Flow('simple-linear-listen').add(
    A(),
    B()
    )

engine = taskflow.engines.load(flow, store = dict(a_msg = 'a', b_msg = 'b'))

engine.notifier.register('*', flow_watch)

engine.notifier.register('*', task_watch)

engine.run()

Flow State:RUNNING
Flow Details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c807790>, 'old_state': 'PENDING', 'flow_name': u'simple-linear-listen', 'flow_uuid': 'd4470d4f-1a17-44ec-9d55-056c3f74f9c5'}
Task state:RUNNING
Task details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c807790>, 'old_state': 'PENDING', 'flow_name': u'simple-linear-listen', 'flow_uuid': 'd4470d4f-1a17-44ec-9d55-056c3f74f9c5'}
A:a
B : b
Flow State:SUCCESS
Flow Details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c807790>, 'old_state': 'RUNNING', 'flow_name': u'simple-linear-listen', 'flow_uuid': 'd4470d4f-1a17-44ec-9d55-056c3f74f9c5'}
Task state:SUCCESS
Task details:{'engine': <taskflow.engines.action_engine.engine.SerialActionEngine object at 0x10c807790>, 'old_state': 'RUNNING', 'flow_name': u'simple-linear-listen', 'flow_uuid': 'd4470d4f-1a17-44ec-9d55-056c3f74f9c5'}


In [4]:
#!/usr/bin/env python
# coding=utf-8

from __future__ import print_function
import taskflow.engines
from taskflow.patterns import linear_flow as lf
from taskflow import task

class A(task.Task):
    def execute(self, a_msg, *args, **kwargs):
        print('A : {}' . format(a_msg))

    def revert(self, a_msg, *args, **kwargs):
        print('A {} revert' . format(a_msg))

class B(task.Task):
    def execute(self, b_msg, *args, **kwargs):
        print('B : {}' . format(b_msg))

    def revert(self, b_msg, *args, **kwargs):
        print('B {} revert' .format(b_msg))

class C(task.Task):
    def execute(self, c_msg, *args, **kwargs):
        print('C : {}' . format(c_msg))
        raise IOError('C IOError')

flow = lf.Flow('simple-linear-listen').add(
    A(),
    B(),
    C()
    )

engine = taskflow.engines.load(flow, store = dict(a_msg = 'a', b_msg = 'b',c_msg = 'c'))
try:
    engine.run()
except Exception as e:
    print("flow failed:{}" .format(e))

A : a
B : b
C : c
B b revert
A a revert
flow failed:C IOError
